In [3]:
import tensorflow as tf
import pandas as pd

In [4]:
from sklearn.preprocessing import LabelEncoder

# Membaca dataset dari file CSV
dataset = pd.read_csv('activities.csv')

# Melakukan label encoding pada kolom activity_name
label_encoder = LabelEncoder()
dataset['activity_code'] = label_encoder.fit_transform(dataset['activity_name'])
dataset

,activity_name,duration,points,activity_code
0,Running,10,20,1
1,Running,20,30,1
2,Running,30,50,1
3,Running,45,70,1
4,Running,60,100,1
5,Walking,15,10,2
6,Walking,30,15,2
7,Walking,45,20,2
8,Walking,60,30,2
9,Walking,90,45,2


In [6]:
from sklearn.model_selection import train_test_split
# Memisahkan fitur dan target
X = dataset.drop(columns=['points', 'activity_name'])
y = dataset['points']

# Memisahkan dataset menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
from tensorflow.keras.layers import Dropout
# Membangun model menggunakan TensorFlow
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),  # Menambahkan Dropout layer
    tf.keras.layers.Dense(64, activation='relu'),
    Dropout(0.2),  # Menambahkan Dropout layer
    tf.keras.layers.Dense(1)
])

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Melatih model
model.fit(X_train, y_train, epochs=50, batch_size=1, verbose=1)

# Evaluasi model
loss = model.evaluate(X_test, y_test)
print("Loss:", loss)


d:\miniconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 3111.1770
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3987.1848 
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1110.6711
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1568.8309 
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1024.4681
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1085.3818 
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 605.3022
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 700.5598 
Epoch 9/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1053.7180 
Epoch 10/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 797.0175  
Epoch 11/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 569.5833 
Epoch 12/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 804.4626 
Epoch 13/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1075.5530 
Epoch 14/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1437.4637
Epoch 15/50
12/12 ━━━━━━━━━━━━━━━━━━━━

In [8]:
X_test

,duration,activity_code
0,10,1
1,20,1
5,15,2
14,90,0


In [9]:

# Prediksi jumlah poin untuk data uji
predictions = model.predict(X_test)
print("Prediksi jumlah poin untuk data uji:", predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Prediksi jumlah poin untuk data uji: [[ 7.889278]
 [15.759493]
 [11.051485]
 [71.486404]]


In [10]:
# Mengubah hasil prediksi menjadi DataFrame
predictions_df = pd.DataFrame(predictions, columns=['predicted_points'])

# Menggabungkan dengan kolom activity_code
predictions_df['activity_code'] = X_test['activity_code'].values

# Mengembalikan label encoding
predictions_df['activity_name'] = label_encoder.inverse_transform(predictions_df['activity_code'])

# Menampilkan hasil
print(predictions_df)

   predicted_points  activity_code activity_name
0          7.889278              1       Running
1         15.759493              1       Running
2         11.051485              2       Walking
3         71.486404              0       Cycling


In [11]:
from sklearn.metrics import mean_absolute_error

# Menghitung Mean Absolute Error (MAE) antara nilai sebenarnya dan prediksi
mae = mean_absolute_error(y_test, predictions)
print("Mean Absolute Error:", mae)


Mean Absolute Error: 11.479077458381653


In [12]:
# Simpan model dalam format HDF5
# model.save('model_activity.h5')
model.save('saved_model.keras')

In [13]:
# Simpan arsitektur model ke dalam format JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


In [14]:
# Simpan parameter bobot model ke dalam format HDF5
model.save_weights("model.weights.h5")
